```
cd ~/GIT/cnv-gene-mapping
sos run dsc/20190717_workflow.ipynb default:1-4 -s build
sos run dsc/20190717_workflow.ipynb get_stats:1-4 -s build
sos run dsc/20190717_workflow.ipynb -s build -j 6
```

In [ ]:
[global]
parameter: cwd = path("~/GIT/github/cnv-gene-mapping/data")
def fmtP(x):
    return str(x).replace(".", "p").replace(' ', '_').replace('"', "").replace("'", "").replace("-", '_')

In [ ]:
[default_1, get_stats_1]
input: f"{cwd:a}/deletion.gz"
output: f'{_input:n}_geneblock.gz'
python: expand = '${ }'
    import pandas as pd
    data = pd.read_table("${_input}", compression = "gzip", sep = "\t", header = 0)
    data = data.rename(columns = {"Unnamed: 0": "gene"})
    all0 = list()
    for i in range(data.shape[0]):
        if sum(data.iloc[i, 1:]) == 0:
            all0.append(i)
    bound = list()
    i = 0; j = 1; n_0 = len(all0)
    while (j < n_0):
        if all0[j] - all0[i] >= 50:
            bound.append(all0[j])
            i = j
        j += 1
    bound = [0] + bound[:-1] + [data.shape[0]]
    fill = list()
    for k in range(len(bound)-1):
        fill_tmp = list()
        if bound[k]+1 > data.shape[0]:
            print (k, bound[k]+1, data.shape[0], "break")
            break
        for l in range(data.shape[1]-1):
            fill_tmp.append(data.iloc[bound[k]:bound[k+1], l+1].tolist())
        fill.append(fill_tmp)
    res = pd.DataFrame(fill) # row: gene block, column: sample name
    res.iloc.to_csv(${_output:r}, compression = "gzip", sep = "\t", header = False, index = False)

In [ ]:
[default_2, get_stats_2]
parameter: sample_size = 10 # sample size: default 100000, test: 1000
parameter: n_batch = 2 # number of simulated sample for each job, default: 200, test: 20
assert sample_size % n_batch == 0
batches = [x+1 for x in range(n_batch)]
input: for_each = ['batches']
output: f'{cwd:a}/simu_data_test/{_input:bnn}.sample.{_batches}.gz'
python: expand = "${ }"
    import pandas as pd, numpy as np
    import random, itertools, ast
    size = int(${sample_size} / ${n_batch})
    random.seed(${_batches})
    data = pd.read_csv(${_input:r}, compression = "gzip", header = None, sep = "\t")
    samples_genome = list()
    for i in range(size):
        order = random.sample(data.columns.tolist(), data.shape[0])
        sample_genome = list(itertools.chain(*list(ast.literal_eval(i) for i in np.diag(data.loc[:, order]))))
        samples_genome.append(sample_genome)
    samples_genome_df = pd.DataFrame(samples_genome) # row: sample name, column: genes
    samples_genome_df.to_csv(${_output:r}, compression = "gzip", sep = "\t", header = False, index = False)

In [ ]:
[default_3, get_stats_3]
input: group_by = 'all'
output: f'{_input[0]:nn}.combined.gz'
bash: expand = "${ }"
    zcat ${_input} | gzip > ${_output}

In [ ]:
[default_4]
parameter: shape = 3
parameter: scale = 1
parameter: penetrance = 0.05
parameter: seed = 999999
parameter: ctrl_case_ratio = 1.0
output: f'{_input:nn}.X.gz', f'{_input:nn}.y.gz', f'{_input:nn}.ycase.index.gz', f'{_input:nn}.yctrl.index.gz'
python: expand = "${ }"
    import pandas as pd, numpy as np
    import random
    np.random.seed(${seed})
    data = pd.read_csv(${_input:r}, compression = "gzip", sep = "\t", header = None)
    d1 = data.values
    beta0 = np.log(${penetrance}/(1-${penetrance}))
    beta1s = [np.log(x) for x in np.random.gamma(${shape}, ${scale}, data.shape[1])] # ORs follow gamma(5,1)
    ys = np.matmul(d1, beta1s) + beta0
    ys_p = [np.exp(i) / (1+np.exp(i)) for i in res.iloc[:, 0]]
    ys_ber = [np.random.binomial(1, x) for x in ys_p]
    res = pd.DataFrame(pd.np.column_stack([ys_ber, res]))
    case = res[res[0] == 1]
    ctrl = res[res[0] == 0].sample(n = int(case.shape[0] * ${ctrl_case_ratio}), random_state = ${seed})
    y_x = case.append(ctrl).sort_index()
    case_index = pd.DataFrame(sorted(case.index.values))
    ctrl_index = pd.DataFrame(sorted(ctrl.index.values))
    y_x.iloc[:, 1:].to_csv(${_output[0]:r}, compression = "gzip", sep = "\t", header = False, index = False)
    y_x.iloc[:, [0]].to_csv(${_output[1]:r}, compression = "gzip", sep = "\t", header = False, index = False)
    case_index.to_csv(${_output[2]:r}, compression = "gzip", sep = "\t", header = False, index = False)
    ctrl_index.to_csv(${_output[3]:r}, compression = "gzip", sep = "\t", header = False, index = False)

In [ ]:
[get_stats_4]
parameter: 
output: f'{_input:n}.blocks.pkl', f'{_input:n}.blocks.histogram.pdf'
python: expand = "${ }"
    import pandas as pd, numpy as np
    from operator import itemgetter
    from itertools import *
    from collections import OrderedDict
    import pickle
    import matplotlib.pyplot as plt
    data = pd.read_csv(${_input:r}, compression = "gzip", sep = "\t", header = None)
    data = data.loc[:, (data != 0).any(axis = 0)]
    indices = list(data.columns)
    groups = []
    for k, g in groupby(enumerate(indices), lambda x: x[0] - x[1]):
        groups.append(list(map(itemgetter(1), g)))
    dfs_dict = OrderedDict()
    i = 1
    for item in groups:
        dfs_dict[f"block{i}"] = data.loc[:, item]
        i += 1
    pickle.dump(dfs_dict, open(${_output[0]:r}, "wb"))
    group_len = [len(item) for item in groups]
    counts = {i: group_len.count(i) for i in set(group_len) if i!=0}
    fig, ax = plt.subplots(figsize = (8,6))
    plt.bar(list(counts.keys()), list(counts.values()), width = 1)
    ax.set_title("Histogram of number of genes in blocks")
    plt.savefig(${_output[1]:r})

In [ ]:
[get_stats_5]
depends: R_library("data.table"), R_library('susieR')
parameter: L = 10
parameter: pve = 0.005
parameter: method = "optim"
suffix = f'SuSiE.L_{L}.prior_{fmtP(pve)}'
print (suffix)
output: f'{_input[0]:nn}.{suffix}.susie.rds'
R: expand = '${ }', stderr = f'{_input[0]:nn}.stderr', stdout = f'{_input[0]:nn}.stdout'
    library(susieR)
    library(data.table)
    X = as.matrix(data.table::fread("zcat ${_input[0]}"))
    y = as.matrix(data.table::fread("zcat ${_input[1]}"))
    storage.mode(X) = 'double'
    storage.mode(y) = 'double'
    res = susie(X, y, L = ${L}, scaled_prior_variance = ${pve}, estimate_prior_method = '${method}')
    saveRDS(res, ${_output:r})

In [1]:
import pandas as pd, numpy as np
data = pd.read_csv("/home/min/GIT/cnv-gene-mapping/dsc/deletion_geneblock.sample.1.gz", sep = "\t", compression = "gzip", header = None)

In [2]:
data = data.iloc[:, 0:5000]

In [3]:
data.shape

(500, 5000)

In [4]:
data = data.loc[:, (data != 0).any(axis = 0)]

In [5]:
data.head()

,30,31,32,33,34,35,36,37,44,45,46,47,48,49,50,51,52,53,194,195,196,197,198,199,200,201,883,1030,1058,1059,1060,1062,1184,1185,1234,1245,1261,1262,1263,1264,...,3268,3269,3270,3271,3272,3273,3274,3309,3319,3320,3321,3718,3719,3720,3721,3833,3949,3961,4300,4301,4344,4345,4410,4411,4433,4505,4507,4508,4509,4510,4633,4634,4635,4778,4806,4824,4859,4942,4943,4957
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
data.iloc[0:3, 0:5]

,30,31,32,33,34
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0


In [7]:
data.loc[:, [30, 31, 32, 33, 34, 35, 36, 37]].head()

,30,31,32,33,34,35,36,37
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0


In [8]:
sum(data.iloc[:, 115].tolist())

14

In [9]:
indices = list(data.columns)

In [10]:
from operator import itemgetter
from itertools import *
groups = []
for k, g in groupby(enumerate(indices), lambda x: x[0] - x[1]):
    groups.append(list(map(itemgetter(1), g)))
print (groups)

[[30, 31, 32, 33, 34, 35, 36, 37], [44, 45, 46, 47, 48, 49, 50, 51, 52, 53], [194, 195, 196, 197, 198, 199, 200, 201], [883], [1030], [1058, 1059, 1060], [1062], [1184, 1185], [1234], [1245], [1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286], [1289, 1290, 1291, 1292, 1293, 1294], [1643, 1644, 1645, 1646, 1647], [1874, 1875, 1876], [2193, 2194], [2275], [2330, 2331, 2332, 2333, 2334, 2335], [2611], [2627], [2632, 2633], [2662, 2663, 2664, 2665], [2708], [2842, 2843], [2899], [2915], [3037, 3038, 3039, 3040, 3041, 3042, 3043, 3044, 3045, 3046, 3047, 3048, 3049, 3050, 3051, 3052], [3056, 3057, 3058, 3059, 3060, 3061, 3062, 3063], [3169, 3170], [3204, 3205], [3221], [3231], [3265, 3266, 3267, 3268, 3269, 3270, 3271, 3272, 3273, 3274], [3309], [3319, 3320, 3321], [3718, 3719, 3720, 3721], [3833], [3949], [3961], [4300, 4301], [4344, 4345], [4410, 4411], [4433], [4505], [4507, 4508, 450

In [11]:
import pickle
from collections import OrderedDict
dfs_dict = OrderedDict()
for item in groups:
    dfs_dict[f"block{item[0]}"] = data.loc[:, item]

In [13]:
pickle.dump(dfs_dict, open("/home/min/GIT/cnv-gene-mapping/dsc/tmp.pkl", "wb"))

In [12]:
dfs_dict["block30"].head()

,30,31,32,33,34,35,36,37
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0


In [14]:
group_len = [len(item) for item in groups]

In [19]:
import matplotlib.pyplot as plt
counts = {i: group_len.count(i) for i in set(group_len) if i!=0}
fig, ax = plt.subplots(figsize = (8,6))
plt.bar(list(counts.keys()), list(counts.values()), width = 0.8)
ax.set_title("Histogram of number of genes in blocks")
plt.savefig("/home/min/GIT/cnv-gene-mapping/dsc/tmp.pdf")
# plt.show()

In [1]:
import pandas as pd, numpy as np
data = pd.read_table("/home/min/GIT/cnv-gene-mapping/data/simu_data_test/deletion_geneblock.sample.X.gz", compression = "gzip", sep = "\t", header = None)
data.shape

(100, 23343)

In [2]:
beta0 = np.log(0.05/(1-0.05))
beta1s = [np.log(x) for x in np.random.gamma(5, 1, data.shape[1])]
np.random.seed(9999999)
ys = [np.around(sum([x*y for x,y in zip(beta1s, data.iloc[i, :].tolist())]) + beta0, 6) for i in range(data.shape[0])]

In [3]:
y_df = pd.DataFrame(ys)

In [4]:
y_df.tail()

,0
95,-2.944439
96,43.804725
97,-2.944439
98,-0.131875
99,-2.944439


In [5]:
ys_1 = np.around(np.matmul(data.values, beta1s) + beta0, 6)
y_df_1 = pd.DataFrame(ys_1)

In [6]:
y_df_1.tail()

,0
95,-2.944439
96,43.804725
97,-2.944439
98,-0.131875
99,-2.944439


In [7]:
df = y_df == y_df_1

In [8]:
sum(df.iloc[:, 0].tolist())

100

In [34]:
susie_res = readRDS("/home/min/GIT/cnv-gene-mapping/data/simu_data_test/deletion_geneblock.sample.SuSiE.L_10.prior_0p005.susie.rds")

In [35]:
names(susie_res)

[1] "alpha"                  "mu"                     "mu2"                   
 [4] "Xr"                     "KL"                     "lbf"                   
 [7] "sigma2"                 "V"                      "pi"                    
[10] "null_index"             "converged"              "elbo"                  
[13] "niter"                  "intercept"              "fitted"                
[16] "sets"                   "pip"                    "X_column_scale_factors"

In [36]:
mean(susie_res$pip)

[1] 0.0004283113

In [37]:
tail(sort(susie_res$pip), 5)

[1] 0.0007206053 0.0007206053 0.0007206053 0.0007206053 0.0007206053

In [38]:
# %put susie_res

In [40]:
susie_res["pip"].index(max(susie_res["pip"]))

In [150]:
for i,j in enumerate(susie_res["pip"]):
    if j == max(susie_res["pip"]):
        print (i)

16007
16008
16009
16010
16011
16012


In [8]:
susie_res.keys()

dict_keys(['alpha', 'mu', 'mu2', 'Xr', 'KL', 'lbf', 'sigma2', 'V', 'pi', 'null_index', 'converged', 'elbo', 'niter', 'intercept', 'fitted', 'sets', 'pip', 'X_column_scale_factors'])

In [153]:
sorted(susie_res["pip"])[-10:]

[0.000665602401835996,
 0.000665783181054036,
 0.000665783181054036,
 0.000665783181054036,
 0.000761617759273969,
 0.000761617759273969,
 0.000761617759273969,
 0.000761617759273969,
 0.000761617759273969,
 0.000761617759273969]

In [154]:
susie_res["fitted"][:10]

[0.622747786415506,
 0.62230394639043,
 0.622833298789739,
 0.62230394639043,
 0.626508087810574,
 0.622663406869125,
 0.623389166799308,
 0.623124755851558,
 0.623217382940279,
 0.62230394639043]

In [155]:
susie_res["sets"].keys()

dict_keys(['cs', 'coverage'])

In [156]:
susie_res["sets"]

{'coverage': 0.95, 'cs': None}

In [26]:
import pandas as pd

In [32]:
d = pd.read_table("/home/min/GIT/cnv-gene-mapping/data/simu_data_test/deletion_geneblock.sample.y.gz", compression = "gzip", sep = "\t", header = None)

In [33]:
d.head()

,0
0,0.226804
1,0.999957
2,0.050000
3,1.000000
4,0.999973


In [34]:
d1 = pd.read_table("/home/min/GIT/cnv-gene-mapping/data/simu_data_test/v1/deletion_geneblock.sample.y.gz", compression = "gzip", sep = "\t", header = None)

In [35]:
d1.head()

,0
0,0.986416
1,0.050000
2,0.999957
3,0.050000
4,1.000000
